In [1]:
import copy

import matplotlib
import numpy as np
from dm_control import viewer
import matplotlib.pyplot as plt
from matplotlib import animation
from tqdm import tqdm
from simulation.dm_control.ddpg.ddpg import DDPGagent, OUNoise
import simulation.dm_control.simulation_control.environments as environments

In [2]:
random_state = np.random.RandomState(42)

SAVE_VIDEOS = False
RESUME_TRAINING = False
PATH_MODEL = 'passive_hand'
NUM_EPISODES = 200
BATCH_SIZE = 128
DURATION = 100
ACTOR_LEARNING_RATE=1e-4
CRITIC_LEARNING_RATE=1e-3
GAMMA=0.99
TAU=1e-2

env = environments.load(domain_name='passive_hand', task_name='lift_sparse')
action_spec = env.action_spec()
dim_action = action_spec.shape[0]
dim_obs = 21

In [3]:
agent = DDPGagent(
    dim_obs,
    dim_action,
    actor_learning_rate=ACTOR_LEARNING_RATE,
    critic_learning_rate=CRITIC_LEARNING_RATE,
    gamma=GAMMA,
    tau=TAU
)
if RESUME_TRAINING: agent.load(PATH_MODEL)
noise = OUNoise(dim_action, action_spec.minimum, action_spec.maximum)

def denorm(a): #  use on model output before passing to env
    act_k = (action_spec.maximum - action_spec.minimum) / 2.
    act_b = (action_spec.maximum + action_spec.minimum) / 2.
    return a * act_k + act_b

def norm(a): # use on env output before passing to model
    act_k_inv = 2. / (action_spec.maximum - action_spec.minimum)
    act_b = (action_spec.maximum + action_spec.minimum) / 2.
    return act_k_inv * (a - act_b)

def parse_obs(obs):
    """
    Take only gripper position and object position???
    """
    x = np.array([])
    for k, v in obs.items():
        if k == 'simulation_time': continue
        x = np.append(x, v)
    return x

In [4]:
def save_video(frames, path, framerate=30):
    height, width, _ = frames[0].shape
    dpi = 70
    orig_backend = matplotlib.get_backend()
    matplotlib.use('Agg')  # Switch to headless 'Agg' to inhibit figure rendering.
    fig, ax = plt.subplots(1, 1, figsize=(width / dpi, height / dpi), dpi=dpi)
    matplotlib.use(orig_backend)  # Switch back to the original backend.
    ax.set_axis_off()
    ax.set_aspect('equal')
    ax.set_position([0, 0, 1, 1])
    im = ax.imshow(frames[0])
    def update(frame):
        im.set_data(frame)
        return [im]
    interval = 1000/framerate
    anim = animation.FuncAnimation(fig=fig, func=update, frames=frames,
                                   interval=interval, blit=True, repeat=False)
    anim.save(f'{path}.mp4', writer='ffmpeg')


In [ ]:
frames = []
rewards = []
avg_rewards = []

for episode in tqdm(range(NUM_EPISODES)):
    frames.clear()

    time_step = env.reset()
    state = parse_obs(time_step.observation)
    noise.reset()
    episode_reward = 0
    episode_reward_history = []
    for step in range(DURATION):
        if SAVE_VIDEOS:
            camera0 = env.physics.render(camera_id=3, height=200, width=200)
            frames.append(np.hstack((camera0,)))


        action = agent.get_action(state)
        action = noise.get_action(action, step)
        try:
            time_step_2 = env.step(denorm(action))
        except:
            print(f'Physics Error: {action}')
            break
        state_2 = parse_obs(time_step_2.observation)
        reward = time_step_2.reward
        agent.memory.push(state, action, reward, state_2, -1)
        state = state_2
        if len(agent.memory) > BATCH_SIZE:
            agent.update(BATCH_SIZE)
        episode_reward += reward
        episode_reward_history.append(reward)
    if SAVE_VIDEOS:
        dest = f'ep{episode}_rew{int(episode_reward)}'
        with open('history.txt', 'a') as f:
            f.write(f'{dest}\n{str(episode_reward_history)}\n')
        save_video(frames, f'ep{episode}_rew{int(episode_reward)}', framerate=1./env.control_timestep())
    print(f"episode: {episode}, "
      f"reward: {np.round(episode_reward, decimals=2)}, "
      f"average_reward: {np.mean(rewards[-10:])}")
    rewards.append(episode_reward)
    avg_rewards.append(np.mean(rewards[-10:]))

agent.save(PATH_MODEL)

plt.plot(rewards)
plt.plot(avg_rewards)
plt.plot()
plt.xlabel('Episode')
plt.ylabel('Reward')
plt.show()

  0%|          | 0/200 [00:00<?, ?it/s]c:\users\leeji\pycharmprojects\algorithms\venv\lib\site-packages\numpy\core\fromnumeric.py:3372: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\users\leeji\pycharmprojects\algorithms\venv\lib\site-packages\numpy\core\_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
  0%|          | 1/200 [00:00<01:13,  2.72it/s]

episode: 0, reward: -59.45, average_reward: nan


  1%|          | 2/200 [00:01<02:56,  1.12it/s]

episode: 1, reward: -76.18, average_reward: -59.445239261050794


  2%|▏         | 3/200 [00:03<03:42,  1.13s/it]

...

episode: 143, reward: -62.64, average_reward: -24584.641643776424


 72%|███████▎  | 145/200 [03:22<01:16,  1.40s/it]

episode: 144, reward: -152.92, average_reward: -24585.210104834947


 73%|███████▎  | 146/200 [03:23<01:15,  1.40s/it]

episode: 145, reward: -96.44, average_reward: -24592.235282506845


In [ ]:
t=-1
def policy(time_step):
    global t
    t += 1
    state = parse_obs(time_step.observation)
    action = agent.get_action(state)
    # print(time_step)
    action = noise.get_action(action, t)
    action = denorm(action)
    # print(action)
    # print(calc_reward(time_step.observation))
    return action

viewer.launch(env, policy=policy)

In [ ]:

#@title Loading and simulating a `suite` task{vertical-output: true}

# Load the environment
# random_state = np.random.RandomState(42)
# env = suite.load('hopper', 'stand', task_kwargs={'random': random_state})

# Simulate episode with random actions
duration = 10  # Seconds
frames = []
ticks = []
rewards = []
observations = []

spec = env.action_spec()
time_step = env.reset()

while env.physics.data.time < duration:
    # action = random_state.uniform(spec.minimum, spec.maximum, spec.shape)
    action = agent.get_action(state)
    action = denorm(action)

    time_step = env.step(action)

    camera0 = env.physics.render(camera_id=3, height=200, width=200)
    # camera1 = env.physics.render(camera_id=3, height=500, width=500)
    frames.append(np.hstack((camera0,)))
    rewards.append(time_step.reward)
    observations.append(copy.deepcopy(time_step.observation))
    ticks.append(env.physics.data.time)

save_video(frames, 'lmao', framerate=1./env.control_timestep())

# Show video and plot reward and observations
# num_sensors = len(time_step.observation)
#
# _, ax = plt.subplots(1 + num_sensors, 1, sharex=True, figsize=(4, 8))
# ax[0].plot(ticks, rewards)
# ax[0].set_ylabel('reward')
# ax[-1].set_xlabel('time')
#
# for i, key in enumerate(time_step.observation):
#   data = np.asarray([observations[j][key] for j in range(len(observations))])
#   ax[i+1].plot(ticks, data, label=key)
#   ax[i+1].set_ylabel(key)